## Evaluation simple encoder transformer
- trained using a small balanced dataset
- tested with original dataset

In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

EVAL = "03_simple_transformer"

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


## Load model

In [2]:
class TransformerClassifier(nn.Module):
    """
    num_heads = more heads capture different attention but increase computation
    num_layers = more make the model deeper but can overfit if too high
    """

    def __init__(self, input_dim, num_heads=4, num_layers=4):
        super().__init__()
        # d_model = input_dim (number of features)
        # TODO: add drop out to avoid overfitting
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads)
        # stacks multiple encoder layers (num_layers controls depth)
        self.encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_layers)
        # linear layer to map the output to a single value (binary classification)
        self.linear_layer = nn.Linear(in_features=input_dim, out_features=1)

    def forward(self, x):
        # Output shape: (batch_size, seq_len, features) or (batch_size, features)
        z = self.encoder(x)
        if z.dim() == 3:  # If (batch_size, seq_len, features), take the last timestep
            # NOTE: last timestep -> the most recent ICU data is usually the most relevant for prediction
            z = z[:, -1, :]
        return self.linear_layer(z)

model = TransformerClassifier(input_dim=38, num_heads=2)
model.load_state_dict(torch.load("./saved/03_simple_transformer.pth"))

/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/venv-sepsis/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

## Testing data

In [3]:
from utils import get_data
from sklearn.model_selection import train_test_split

tensor_ds_path = f"{project_root}/dataset/small_dataset_tensors.pth"
if os.path.exists(tensor_ds_path):
    data = torch.load(tensor_ds_path)
    print("Loaded saved dataset tensors")
    X_tensor = data["X_test"] 
    y_tensor = data["y_test"]


Loaded saved dataset tensors


## Testing loop

In [4]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torchmetrics import Accuracy

batch_size = 512 
test_dataset = TensorDataset(X_tensor, y_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
test_loss, test_acc = 0, 0
loss_fn = nn.BCEWithLogitsLoss()
t_accuracy = Accuracy(task='binary')
all_y_logits, all_y_probs, all_y_pred, all_y_test = [], [], [], []

with torch.inference_mode():
    progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for X_batch, y_batch in progress_bar:
        y_logits = model(X_batch)
        y_probs = torch.sigmoid(y_logits)
        y_pred = torch.round(y_probs)
        
        loss = loss_fn(y_logits, y_batch.unsqueeze(1).float())
        acc = t_accuracy(y_pred, y_batch.unsqueeze(1).float())
        
        test_loss += loss.item()
        test_acc += acc.item()
        
        progress_bar.set_postfix({"Loss": loss.item(), "Acc": acc.item()})
        
        all_y_logits.append(y_logits.cpu())
        all_y_probs.append(y_probs.cpu())
        all_y_pred.append(y_pred.cpu())
        all_y_test.append(y_batch.cpu())

test_loss /= len(test_loader)
test_acc /= len(test_loader)

print(f"Test Loss: {test_loss:.5f} | Test Accuracy: {test_acc:.2f}%")

Test Loss: 0.50726 | Test Accuracy: 0.78%


In [5]:
import pandas as pd
import numpy as np
from t_utils import save_eval_csv

all_y_logits = torch.cat(all_y_logits).numpy().flatten()
all_y_probs = torch.cat(all_y_probs).numpy().flatten()
all_y_pred = torch.cat(all_y_pred).numpy().flatten()
all_y_test = torch.cat(all_y_test).numpy().astype(int) 

df = pd.DataFrame({
    'y_logits': all_y_logits,
    'y_probs': all_y_probs,
    'y_pred': all_y_pred,
    'y_test': all_y_test
})


In [6]:
save_eval_csv(df, EVAL)

In [7]:
accuracy = (df['y_pred'] == df['y_test']).mean() * 100
print(f"Total Accuracy: {accuracy:.2f}%")

Total Accuracy: 77.69%


## Metrics and plots
- If false negatives are worse → Use best threshold (F1-optimized).
- If false positives are worse → Use precision-optimized threshold.
- If both are critical → Stick to best_threshold but adjust slightly based on results.

In [12]:
from t_utils import save_plots, FEATURE_NAMES, save_metrics

y_test = df['y_test'].values
y_probs = df['y_probs'].values
y_pred = df['y_pred'].values

save_plots(
    y_test=y_test,
    y_probs=y_probs,
    y_pred=y_pred,
    model=model, 
    feature_names=FEATURE_NAMES,
    eval=EVAL,
    attention_weights=[]
)

save_metrics(
    y_test=y_test,
    y_probs=y_probs,
    y_pred=y_pred,
    eval=EVAL,
)

TypeError: Object of type float32 is not JSON serializable